In [1]:
%cd /opt/ml/project/team/code

/opt/ml/project/team/code


In [59]:
import torch
import json
import argparse
import os
import json
from attrdict import AttrDict
import sys
from torch.functional import split
import yaml
import torch
from torch.utils.data import DataLoader, SequentialSampler
from torch.cuda.amp import autocast
from tqdm.auto import tqdm
import copy
from data_utils import WOSDataset
from prepare import get_data, get_stuff, get_model
import parser_maker
from data_utils import split_slot, convert_state_dict

from training_recorder import RunningLossRecorder

In [3]:
!pwd

/opt/ml/project/team/code


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# conf=dict()
# with open(config_root) as f:
#     conf = yaml.load(f, Loader=yaml.FullLoader)

task_dir = '/opt/ml/project/team/code/results/random_1125_1'
# eval_data = json.load(open(f"/opt/ml/input/data/eval_dataset/eval_dials.json", "r"))

config = json.load(open(f"{task_dir}/exp_config.json", "r"))

# print(config)
config = argparse.Namespace(**config)
_, slot_meta, ontology = get_data(config)

config.device = device

config.use_cache_examples2features = False
tokenizer, processor, _, _ = get_stuff(config,
             None, None, slot_meta, ontology)

# eval_data = WOSDataset(eval_features)
# eval_sampler = SequentialSampler(eval_data)
# eval_loader = DataLoader(
#     eval_data,
#     batch_size=8,
#     sampler=eval_sampler,
#     collate_fn=processor.collate_fn,
# )
# print("# eval:", len(eval_data))

# print(slot_meta)

model =  get_model(config, tokenizer, ontology, slot_meta)


ckpt = torch.load(f"{task_dir}/model-best.bin", map_location="cpu")
# ckpt = torch.load("/opt/ml/gyujins_file/model-best.bin", map_location="cpu")

model.load_state_dict(ckpt)
model.to(device)
print('done')

using train: train_dials.json



done


In [5]:
import pickle

file_name = '/opt/ml/input/data/tmp/good_ones_size_1248_0.pkl'
data = pickle.load(open(file_name, 'rb'))

In [6]:
from data_utils import DSTInputExample
from inference import postprocess_state

In [7]:
datum = data[0]
datum

AttrDict({'sys': '출발 시간과 도착 시간을 알 수 있을까요?', 'usr': '오동통규동동에서 출발해서 오동통규동동에 2시 22분에 도착해야해요.', 'mod_labels': ['택시-도착 시간-02:22', '택시-출발지-오동통규동동'], 'ori_usr': '친구가 노원역에 22시10분까지 오라고 했는데 거기에 출발 시간을 맞춰주심 될 꺼 같아요.', 'ori_labels': ['택시-도착 시간-22:10', '택시-출발 시간-dontcare']})

In [12]:
examples = []
for idx, datum in enumerate(data):
    examples.append(
            DSTInputExample(
                guid=f"need_guid_{idx}",
                context_turns=["", ""],
                current_turn=[datum.sys, datum.usr],  # TRADE,SUMBT 다름 유의
                label=datum.mod_labels, 
                before_label={}, # This was for som_dst
            )
    )

In [13]:
features = processor.convert_examples_to_features(examples)

In [14]:
ds = WOSDataset(features)
sampler = SequentialSampler(ds)
eval_loader = DataLoader(
    ds,
    batch_size=8,
    sampler=sampler,
    collate_fn=processor.collate_fn,
)

In [15]:
use_amp = False

model.eval()
predictions = {}
pbar = tqdm(eval_loader, total=len(eval_loader), file=sys.stdout)

for batch in pbar:
    input_ids, segment_ids, input_masks, gating_ids, target_ids, guids = [
        b.to(device) if not isinstance(b, list) else b for b in batch
    ]

    with torch.no_grad():
        with autocast(enabled=use_amp):
            o, g = model(input_ids, segment_ids, input_masks, 9)

        _, generated_ids = o.max(-1)
        _, gated_ids = g.max(-1)


    for guid, gate, gen in zip(guids, gated_ids.tolist(), generated_ids.tolist()):
        prediction = processor.recover_state(gate, gen)
        prediction = postprocess_state(prediction)
        predictions[guid] = prediction
pbar.close()


In [ ]:
def 

In [69]:
good = []
bad = []
for example in examples:
    guid = example.guid

    answer = convert_state_dict(example.label)
    pred = convert_state_dict(predictions[guid])
    if set(pred).issubset(set(answer)):
        good.append(
            AttrDict(
                example=example,
                prediction=predictions[guid],
            )
        )
    else:
        bad.append(
            AttrDict(
                example=example,
                prediction=predictions[guid],
            )
        )
    
len(good), len(bad)

(646, 602)

In [92]:
bad[idx].example

DSTInputExample(guid='need_guid_39', context_turns=['', ''], current_turn=['선호하시는 택시 종류가 있으신가요?', '개인택시로 찾아주세요. 도착은 00:33에 해야합니다.'], label=('택시-종류-개인', '택시-도착 시간-00:33'), before_label={})

In [97]:
idx = 0

In [118]:
print(f'SYS: {bad[idx].example.current_turn[0]}')
print(f'USR: {bad[idx].example.current_turn[1]}')
print(f' ANS: {sorted(bad[idx].example.label)}')
print(f'PRED: {sorted(bad[idx].prediction)}')
print(bad[idx].example.guid)
idx += 1

SYS: 그럼 코엑스 아쿠아리움은 어떨까요? 동물원으로 분류가 되고 평점도 좋은 4점입니다.
USR: 네 동물원이 좋겠네요. 그리고 잠실 야구 경기장과 같은 공원을 찾아주세요.
 ANS: ['관광-이름-잠실 야구 경기장', '관광-종류-공원']
PRED: ['관광-이름-코엑스 아쿠아리움', '관광-종류-공원', '관광-지역-서울 중앙']
need_guid_35


In [99]:
idx = 0

In [136]:
print(f'SYS: {good[idx].example.current_turn[0]}')
print(f'USR: {good[idx].example.current_turn[1]}')
print(f' ANS: {sorted(good[idx].example.label)}')
print(f'PRED: {sorted(good[idx].prediction)}')
idx += 1

SYS: 별마당 도서관의 영업시간은 10시부터 18시이며 가까운 지하철역은 삼성동중앙역으로 확인됩니다. 지하철역에서 관광지까지 도보로 약 4분 소요됩니다.
USR: 그렇군요. 잘 알겠습니다. 저 그리고 서울 중앙에 위치한 가로수길도 방문해보려고 하는데요. 영업시간과 가까운 지하철역이 어떻게 되
 ANS: ['관광-이름-가로수길', '관광-지역-서울 중앙']
PRED: ['관광-이름-가로수길']


In [137]:
good[0]

AttrDict({'example': DSTInputExample(guid='need_guid_1', context_turns=['', ''], current_turn=['알려주신 조건으로 두 곳 다 예약 가능하십니다. ', '그럼 제이 호텔로 할게요. 비싼 가격대로는 어디가 좋을까요?'], label=('숙소-이름-제이 호텔', '숙소-가격대-비싼'), before_label={}), 'prediction': ['숙소-가격대-비싼', '숙소-이름-제이 호텔']})

In [147]:
diff_good = []
for x in good:
    if set(x.example.label) != set(x.prediction):
        diff_good.append(x)

In [148]:
len(diff_good)

174

In [149]:
idx = 0

In [164]:
print(f'SYS: {diff_good[idx].example.current_turn[0]}')
print(f'USR: {diff_good[idx].example.current_turn[1]}')
print(f' ANS: {sorted(diff_good[idx].example.label)}')
print(f'PRED: {sorted(diff_good[idx].prediction)}')
idx += 1

SYS: 
USR: 안녕하세요. 홀럭키하우스라는 이름의 비싼 가격대의 중식당을 방문하려고 하는데요.
 ANS: ['식당-가격대-비싼', '식당-이름-홀럭키하우스', '식당-종류-중식당']
PRED: ['식당-가격대-비싼', '식당-종류-중식당']


In [ ]:
1200 -> 600 -> 174